In [28]:
#### Gather, All colleg hockey roster data for analysis

url = 'https://www.collegehockeynews.com/reports/roster/Michigan-State/32'

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

## Load results for season to get team links
# Read the CSV file
results = pd.read_csv('..\\TEMP\\current_season.csv')

# Concatenate the unique values in 'Home_Team_Link' and 'Away_Team_Link'
team_links = pd.concat([results['Home_Team_Link'], results['Away_Team_Link']])

# Drop duplicates and NaN values
team_links = team_links.drop_duplicates().dropna()

# Clean up the links - drop everything before the third slash - keep the team name and the school ID as string
team = team_links.str.split('/', expand=True)[3]
number = team_links.str.split('/', expand=True)[4]
# reconstruct the link and store in a new column
team_links = 'https://www.collegehockeynews.com/reports/roster/' + team + '/' + number

# reset the index
team_links = team_links.reset_index(drop=True)






team_links

team[0]

# print the first link
# print(team_links[0])


# results.head()

'Vermont'

In [29]:
# Updated function to parse the college hockey roster HTML table and perform transformations

# Function to split "Last Team" into "Team" and "League" with edge case handling
def split_last_team(last_team):
    # Use regular expression to extract team and league
    match = re.search(r'(.+) \((.+)\)', last_team)
    if match:
        return match.groups()
    else:
        # If no league is specified, return the team as is and leave league blank
        return last_team, ""

# Updated function to correctly capture the player's position and handle edge cases in "Last Team"
def parse_and_transform_roster(html_content):
    # Initialize BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the table with the roster
    roster_table = soup.find('table', {'id': 'players'})
    
    # Extract headers
    header_row = roster_table.find('thead').find('tr')
    headers = [header.text.strip() for header in header_row.find_all('th')]
    headers.append('Position')  # Add the Position column to headers
    
    # Initialize data list and current_position variable
    data = []
    current_position = None  # Initialize as None to later filter out irrelevant rows
    
    # Iterate through each row in the table
    for row in roster_table.find_all('tr'):
        if 'class' in row.attrs and 'stats-section' in row.attrs['class']:
            current_position = row.text.strip()
        else:
            cells = row.find_all('td')
            if cells and current_position:
                row_data = [cell.text.strip() for cell in cells]
                row_data.append(current_position)  # Add the current position to the row data
                data.append(row_data)
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)
    
    # Cleanup: Remove rows where 'No.' column is not numeric
    df = df[df['No.'].str.isnumeric()]
    
    # Cleanup: Drop the 'Pos' column
    df.drop(columns=['Pos'], inplace=True)
    
    # Transform Height to Inches
    df['Height_Inches'] = df['Ht.'].apply(convert_to_inches)

    # Transform NHL Draft to Draft_Year, NHL_Team, and D_Round
    df['Draft_Year'], df['NHL_Team'], df['D_Round'] = zip(*df['NHL Draft'].apply(split_nhl_draft))
    df.drop(columns=['NHL Draft'], inplace=True) # Drop the original NHL Draft column
    
    # Handle edge cases in "Last Team" to split into "Team" and "League"
    df['Team'], df['League'] = zip(*df['Last Team'].apply(split_last_team))
    df.drop(columns=['Last Team'], inplace=True)
    
    # Rename the trouble column Hometown\nLast Team\nNHL Draft
    df.rename(columns={'Hometown\nLast Team\nNHL Draft': 'Hometown'}, inplace=True)

    # assign data types No. Wt. and Height_Inches to int, DOB to datetime
    int_list = ['No.', 'Wt.', 'Height_Inches']
    # Convert columns to numeric, coercing errors to NaN
    df[int_list] = df[int_list].apply(pd.to_numeric, errors='coerce')

    # Replace NaNs with a default value for specific columns
    default_value = 0
    df[int_list] = df[int_list].fillna(value=default_value)

    # Confirm that NaNs are filled
    print(df[int_list].isna().sum())  # Should output all zeros

    # Convert the columns to integers
    df[int_list] = df[int_list].astype(int)

    # df = df[df['Height_Inches'].notna()]  # Assuming convert_to_inches returns None for bad values

    
    return df

# Function to split "NHL Draft" into "Draft_Year", "NHL_Team", and "D_Round"
def split_nhl_draft(nhl_draft):
    try:
        draft_year, nhl_team, d_round = nhl_draft.split('-')
        return draft_year, nhl_team, d_round
    except ValueError:
        # Handle missing or incomplete data
        return None, None, None

# # Test the function
# test_values = ['2022-WSH-7', '', '2021-DET']
# [split_nhl_draft(val) for val in test_values]



# Function to convert height in "ft-in" format to total inches
def convert_to_inches(height_str):
    try:
        feet, inches = map(int, height_str.split('-'))
        return (feet * 12) + inches
    except ValueError:
        return None

In [30]:
## Loop through the team links and parse the roster data 
### Notes: save the roster link to the dataframe and add the team name and school ID

roster_dfs = []  # Assuming this list exists to store each roster DataFrame

# Extract team names from team_links
team_names = pd.Series(team_links).str.split('/', expand=True)[5]

# Loop through each team link
for i, link in enumerate(team_links):
    print(f'Processing team {i+1} of {len(team_links)}')
    
    # Make GET request to team link
    r = requests.get(link)
    
    # Parse the HTML content
    html_content = r.text
    
    # Parse and transform the roster data
    roster_df = parse_and_transform_roster(html_content)
    
    # Reset the index if it's not unique
    roster_df.reset_index(drop=True, inplace=True)
    
    # Add the team name and school ID
    current_team = team_names.iloc[i]  # Extract the current team name
    roster_df['School'] = current_team  # Add it to the DataFrame
    # roster_df['School_ID'] = number[i]  # Assuming number is a list containing school IDs
    # rename Team to Last Team
    roster_df.rename(columns={'Team': 'Last Team'}, inplace=True)
    
    # Add the roster DataFrame to the list
    roster_dfs.append(roster_df)



Processing team 1 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 2 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 3 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 4 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 5 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 6 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 7 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 8 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 9 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 10 of 57
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 11 of 57
No.              0
Wt.            

In [31]:
####### Check the Resulting DataFrames

# Check the number of DataFrames in the list
print(len(roster_dfs)) 
## Show the head of the first DataFrame
# roster_dfs[0].head()

# Create a single master DataFrame and output a CSV file
master_roster_df = pd.concat(roster_dfs)

# Reset the index
master_roster_df.reset_index(drop=True, inplace=True)

# Output the CSV file
master_roster_df.to_csv('..\\TEMP\\2023_master_roster.csv', index=False)


57


In [33]:
## Examine the master DataFrame

# Check the number of rows and columns
print(master_roster_df.shape)

# Check the data types
print(master_roster_df.dtypes)

# Check the first few rows
master_roster_df.head()

(1604, 16)
                 object
No.               int32
Name             object
Yr.              object
Ht.              object
Wt.               int32
DOB              object
Hometown         object
Position         object
Height_Inches     int32
Draft_Year       object
NHL_Team         object
D_Round          object
Last Team        object
League           object
School           object
dtype: object


,,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,School
0,,27,"Arvedson, Adam",Jr,5-11,165,8/30/2001,"Karlstad, Sweden",Defensemen,71,None,None,None,Färjestad BK J20,,Merrimack
1,,37,"Bookman, Zach",So,5-10,178,3/29/2002,"Syracuse, N.Y.",Defensemen,70,None,None,None,Brooks,AJHL,Merrimack
2,,4,"Brown, Mike",Jr,6-2,202,4/3/2001,"Belmont, Mass.",Defensemen,74,None,None,None,Youngstown,USHL,Merrimack
3,,8,"Dennison, Liam",Gr,5-9,190,2/7/1999,"Manotick, Ont.",Defensemen,69,None,None,None,Youngstown,USHL,Merrimack
4,,23,"Djurasevic, Frank",Fr,6-2,198,3/9/2002,"New Rochelle, N.Y.",Defensemen,74,None,None,None,Penticton,BCHL,Merrimack


In [35]:
######### Simple Map of Hometowns

from geopy.geocoders import Nominatim
import geopandas as gpd
import matplotlib.pyplot as plt

df = master_roster_df

# Initialize geocoder
geolocator = Nominatim(user_agent="hockey_hometowns")

# Geocode hometowns to get latitude and longitude
df['Coordinates'] = df['Hometown'].apply(geolocator.geocode).apply(lambda loc: tuple(loc.point) if loc else None)

# Split coordinates into latitude and longitude
df[['Latitude', 'Longitude', '_']] = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Livonia%2C+Mich.&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))